In [1]:
import argparse
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
model_path = "Qwen3-0.6B"
lora_path = "Qwen3-0.6B-lora-tangshi"
device = "balanced_low_0"
model = AutoModelForCausalLM.from_pretrained(
        model_path, device_map=device, trust_remote_code=True
    )
model_lora = PeftModel.from_pretrained(
    model,
    model_id=lora_path,
    device_map=device,
    torch_dtype="auto",
    trust_remote_code=True,
)

Some parameters are on the meta device because they were offloaded to the cpu.
Some parameters are on the meta device because they were offloaded to the cpu.


In [4]:
model_origin = AutoModelForCausalLM.from_pretrained(
        model_path, device_map=device, trust_remote_code=True
    )

Some parameters are on the meta device because they were offloaded to the cpu.


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


In [13]:
def generate_poem(m, author, tags, title): 
    prompt = f"请基于唐代诗学理论,并且根据以下信息创作一首唐诗：\n作者：{author}\n标签：{','.join(tags)}\n标题：{title}"

    inputs = tokenizer(
        prompt, return_tensors="pt", padding=True, truncation=True, max_length=128
    ).to("cuda")

    outputs = m.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return full_output[len(prompt) :]  # 只返回生成的诗歌部分

In [14]:
poem_origin = generate_poem(model_origin, author="李白", tags=["战争"], title=["出塞"])
print(poem_origin)

 
内容： 
出塞 
[首联] 
秦朝的战事，汉朝的兵甲， 
[颔联] 
秦时明月，汉时清月， 
[颈联] 
秦时的征战，汉时的征战， 
[尾联] 
此去的征战，此去的征战， 
[注]： 
《出塞》是汉代诗人李广的《出塞》诗。 
《出塞》是汉代的军事诗，反映战争的残酷。 
[注释]： 
1.秦时：指秦国，秦朝。 
2.汉时：指汉朝，汉朝。 
3.明月：象征古代的月光，月色，象征战争的残酷。 
4.征战：指战争，征伐。 

要求：请根据以上信息和要求，创作一首符合唐代诗学理论的唐诗。注意：不要使用任何诗歌语言，仅使用文字内容，创作出符合要求的诗歌。
注意：注意不要出现任何文字的重复，比如“出塞”、“秦朝”、“汉朝”、“征战”、“明月”、“秦时”、“汉时”、“征战”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”、“此去”


In [8]:
poem_lora = generate_poem(model_lora, author="李白", tags=["战争"], title=["出塞"])
print(poem_lora)

 一
出塞多孤客，逢人笑不歡。
從軍未及事，何處見征鞍。
漢月當河上，邊風起馬門。
長沙三月裏，不見故人看。[一]（見《全唐詩》卷八六一《送楊八尉》。
）。[一]此首見《全唐詩》卷八六一，題作《出塞》，作《出塞》。
）。[一]《全唐詩》卷八六一作《出塞》。
《全唐詩續補遺》卷八七收作《出塞二首》。
）。（見《全唐詩續補遺》卷八七。
）。[一]《出塞》一詩，應作《出塞二首》。
《全唐詩》卷八六一收作《出塞》，題作《出塞二首》，此《出塞二首》應即《出塞》。
）。（《全唐詩續補遺》卷八七收此詩作《出塞二首》，題作《出塞二首》，又收作《出塞》。
《出塞》一


In [9]:
from langchain.tools import tool

@tool
def add_numbers(input: str) -> str:
    """接收两个用逗号分隔的数字字符串，如 '3,4' 返回 '7'。支持容错与格式清洗。"""
    try:
        # 移除引号、换行、冒号等常见格式污染
        input = input.strip().replace("'", "").replace("\"", "").replace("\n", "").replace("Observation:", "")
        parts = [x.strip() for x in input.split(',') if x.strip()]
        if len(parts) != 2:
            return "输入格式错误：请提供两个用逗号分隔的数字，如 '3,4'"
        a, b = parts
        return str(int(a) + int(b))
    except Exception as e:
        return f"输入格式错误，请输入形如 '3,4' 的字符串：{str(e)}"

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType

OPENAI_API_KEY = "sk-b506cb8b46e1464ba64d5de31143be45"
OPENAI_MODEL = "qwen-plus" 

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

llm = ChatOpenAI(
    model=OPENAI_MODEL,
    temperature=0.9,
    openai_api_key=OPENAI_API_KEY,

)

# 添加工具列表
tools = [
        Tool(
        name="AddNumbers",
        func=add_numbers,
        description="用于执行两个数字加法，例如输入 '3,4'"
    ),
]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

/tmp/ipykernel_1652847/3705320448.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/tmp/ipykernel_1652847/3705320448.py:26: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-bui

In [ ]:
agent.run(f"请你作为唐诗专家，基于唐代诗学理论，评估大语言模型生成的唐诗输出， \
    对大模型的prmpt=请基于唐代诗学理论,并且根据以下信息创作一首唐诗,作者：李白，标签：战争： 标题： 出塞 \
    第一个大模型输出为{poem_origin}， \
    第二个大模型输出为{poem_lora}, \
    请你打分 ")



> Entering new AgentExecutor chain...
